# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 21 2022 10:57AM,253588,10,MSP216961,"Methapharm, Inc.",Released
1,Dec 21 2022 10:57AM,253588,10,MSP216962,"Methapharm, Inc.",Released
2,Dec 21 2022 10:57AM,253588,10,MSP216965,"Methapharm, Inc.",Released
3,Dec 21 2022 10:57AM,253588,10,MSP216967,"Methapharm, Inc.",Released
4,Dec 21 2022 10:56AM,253587,10,7370178,"Citieffe, Inc.",Released
5,Dec 21 2022 10:41AM,253585,10,12345625,"Citieffe, Inc.",Released
6,Dec 21 2022 10:39AM,253586,20,ATR-I000035,"HVL, LLC dba Atrium Innovations",Released
7,Dec 21 2022 10:39AM,253586,20,ATR-I000036,"HVL, LLC dba Atrium Innovations",Released
8,Dec 21 2022 10:38AM,253584,19,STR12212022-3,ACG North America LLC,Released
9,Dec 21 2022 10:35AM,253583,12,HH8595,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,253584,Released,1
28,253585,Released,1
29,253586,Released,2
30,253587,Released,1
31,253588,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
253584,NaN,1.0
253585,NaN,1.0
253586,NaN,2.0
253587,NaN,1.0
253588,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
253494,5.0,37.0
253495,0.0,19.0
253497,13.0,2.0
253502,1.0,1.0
253505,0.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
253494,5,37
253495,0,19
253497,13,2
253502,1,1
253505,0,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,253494,5,37
1,253495,0,19
2,253497,13,2
3,253502,1,1
4,253505,0,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,253494,5,37
1,253495,,19
2,253497,13,2
3,253502,1,1
4,253505,,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 21 2022 10:57AM,253588,10,"Methapharm, Inc."
4,Dec 21 2022 10:56AM,253587,10,"Citieffe, Inc."
5,Dec 21 2022 10:41AM,253585,10,"Citieffe, Inc."
6,Dec 21 2022 10:39AM,253586,20,"HVL, LLC dba Atrium Innovations"
8,Dec 21 2022 10:38AM,253584,19,ACG North America LLC
9,Dec 21 2022 10:35AM,253583,12,Hush Hush
10,Dec 21 2022 10:34AM,253582,12,Hush Hush
11,Dec 21 2022 10:34AM,253581,10,"CLINUVEL, Inc."
12,Dec 21 2022 10:18AM,253578,16,TASA USA Inc.
13,Dec 21 2022 10:00AM,253575,16,Sartorius Stedim Filters Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 21 2022 10:57AM,253588,10,"Methapharm, Inc.",,4
1,Dec 21 2022 10:56AM,253587,10,"Citieffe, Inc.",,1
2,Dec 21 2022 10:41AM,253585,10,"Citieffe, Inc.",,1
3,Dec 21 2022 10:39AM,253586,20,"HVL, LLC dba Atrium Innovations",,2
4,Dec 21 2022 10:38AM,253584,19,ACG North America LLC,,1
5,Dec 21 2022 10:35AM,253583,12,Hush Hush,,1
6,Dec 21 2022 10:34AM,253582,12,Hush Hush,,1
7,Dec 21 2022 10:34AM,253581,10,"CLINUVEL, Inc.",,1
8,Dec 21 2022 10:18AM,253578,16,TASA USA Inc.,,1
9,Dec 21 2022 10:00AM,253575,16,Sartorius Stedim Filters Inc.,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 21 2022 10:57AM,253588,10,"Methapharm, Inc.",4,
1,Dec 21 2022 10:56AM,253587,10,"Citieffe, Inc.",1,
2,Dec 21 2022 10:41AM,253585,10,"Citieffe, Inc.",1,
3,Dec 21 2022 10:39AM,253586,20,"HVL, LLC dba Atrium Innovations",2,
4,Dec 21 2022 10:38AM,253584,19,ACG North America LLC,1,
5,Dec 21 2022 10:35AM,253583,12,Hush Hush,1,
6,Dec 21 2022 10:34AM,253582,12,Hush Hush,1,
7,Dec 21 2022 10:34AM,253581,10,"CLINUVEL, Inc.",1,
8,Dec 21 2022 10:18AM,253578,16,TASA USA Inc.,1,
9,Dec 21 2022 10:00AM,253575,16,Sartorius Stedim Filters Inc.,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 21 2022 10:57AM,253588,10,"Methapharm, Inc.",4,
1,Dec 21 2022 10:56AM,253587,10,"Citieffe, Inc.",1,
2,Dec 21 2022 10:41AM,253585,10,"Citieffe, Inc.",1,
3,Dec 21 2022 10:39AM,253586,20,"HVL, LLC dba Atrium Innovations",2,
4,Dec 21 2022 10:38AM,253584,19,ACG North America LLC,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 21 2022 10:57AM,253588,10,"Methapharm, Inc.",4.0,NaN
1,Dec 21 2022 10:56AM,253587,10,"Citieffe, Inc.",1.0,NaN
2,Dec 21 2022 10:41AM,253585,10,"Citieffe, Inc.",1.0,NaN
3,Dec 21 2022 10:39AM,253586,20,"HVL, LLC dba Atrium Innovations",2.0,NaN
4,Dec 21 2022 10:38AM,253584,19,ACG North America LLC,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 21 2022 10:57AM,253588,10,"Methapharm, Inc.",4.0,0.0
1,Dec 21 2022 10:56AM,253587,10,"Citieffe, Inc.",1.0,0.0
2,Dec 21 2022 10:41AM,253585,10,"Citieffe, Inc.",1.0,0.0
3,Dec 21 2022 10:39AM,253586,20,"HVL, LLC dba Atrium Innovations",2.0,0.0
4,Dec 21 2022 10:38AM,253584,19,ACG North America LLC,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4310369,105.0,15.0
12,507165,2.0,0.0
15,506989,56.0,5.0
16,507153,2.0,0.0
19,1014282,89.0,3.0
20,253586,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4310369,105.0,15.0
1,12,507165,2.0,0.0
2,15,506989,56.0,5.0
3,16,507153,2.0,0.0
4,19,1014282,89.0,3.0
5,20,253586,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,105.0,15.0
1,12,2.0,0.0
2,15,56.0,5.0
3,16,2.0,0.0
4,19,89.0,3.0
5,20,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,105.0
1,12,Released,2.0
2,15,Released,56.0
3,16,Released,2.0
4,19,Released,89.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20
Status,,,,,,
Executing,15.0,0.0,5.0,0.0,3.0,0.0
Released,105.0,2.0,56.0,2.0,89.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20
0,Executing,15.0,0.0,5.0,0.0,3.0,0.0
1,Released,105.0,2.0,56.0,2.0,89.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20
0,Executing,15.0,0.0,5.0,0.0,3.0,0.0
1,Released,105.0,2.0,56.0,2.0,89.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()